In [1]:
import pandas as pd

result_path = './s_data_list/final_count_1.csv'
df = pd.read_csv(result_path)

In [2]:
# 'key' 칼럼을 인덱스로 설정
df.set_index('ngram', inplace=True)

In [3]:
k = 0.5
df['P(word|P)'] = (df['P_count'] + k )/ (df['P_count'].sum() + 2 * k) 
df['P(word|N)'] = (df['N_count'] + k )/ (df['N_count'].sum() + 2 * k) 
df 

,P_count,N_count,P(word|P),P(word|N)
ngram,,,,
약세,506,307,0.001967,0.001418
영향,884,667,0.003436,0.003078
인하,1742,1777,0.006769,0.008195
진행,412,229,0.001602,0.001058
되,8059,6489,0.031306,0.029921
...,...,...,...,...
대환,32,14,0.000126,0.000067
코로나,333,407,0.001295,0.001879
지사,31,7,0.000122,0.000035


In [4]:
import numpy as np 
df['log(P(word|P))'] = np.log(df['P(word|P)'])
df['log(P(word|N))'] = np.log(df['P(word|N)']) 
df 

,P_count,N_count,P(word|P),P(word|N),log(P(word|P)),log(P(word|N))
ngram,,,,,,
약세,506,307,0.001967,0.001418,-6.231018,-6.558666
영향,884,667,0.003436,0.003078,-5.673519,-5.783602
인하,1742,1777,0.006769,0.008195,-4.995466,-4.804178
진행,412,229,0.001602,0.001058,-6.436306,-6.851238
되,8059,6489,0.031306,0.029921,-3.463935,-3.509200
...,...,...,...,...,...,...
대환,32,14,0.000126,0.000067,-8.977302,-9.612992
코로나,333,407,0.001295,0.001879,-6.648899,-6.277100
지사,31,7,0.000122,0.000035,-9.008554,-10.272238


In [11]:
words = [index_value for index_value in df.index]

# log 활용 underflow 방지
final_P = np.log(df['P_count'].sum() / (df['P_count'].sum() + df['N_count'].sum()) )
final_N = np.log(df['N_count'].sum() / (df['P_count'].sum() + df['N_count'].sum()) )

scores = []
for token in words:
    final_P += df['log(P(word|P))'][token]
    final_N += df['log(P(word|N))'][token]
    final_P = np.exp(final_P)
    final_N = np.exp(final_N)
    a = final_P / (final_P + final_N)
    b = final_N / (final_P + final_N)
    scores.append(a / b)
    # final_P = np.log(df['P_count'].sum() / (df['P_count'].sum() + df['N_count'].sum()) )
    # final_N = np.log(df['N_count'].sum() / (df['P_count'].sum() + df['N_count'].sum()) )

df['Polarity'] = scores

In [12]:
for word,pola in zip(df.index,df['Polarity']):
    if pola > 1.3 or pola < 0.77:
        pass
    else :
        df = df.drop(word)

In [13]:
df

,P_count,N_count,P(word|P),P(word|N),log(P(word|P)),log(P(word|N)),Polarity
ngram,,,,,,,
약세,506,307,0.001967,0.001418,-6.231018,-6.558666,1.778604
진행,412,229,0.001602,0.001058,-6.436306,-6.851238,1.515001
강세,563,355,0.002189,0.001639,-6.124375,-6.413616,1.336142
지속,970,579,0.003770,0.002672,-5.580731,-5.924975,1.411702
인상,2515,1558,0.009771,0.007186,-4.628315,-4.935662,1.361312
...,...,...,...,...,...,...,...
케이뱅크,47,55,0.000185,0.000256,-8.597812,-8.270758,0.721083
토스,45,28,0.000177,0.000131,-8.640830,-8.937237,1.344922
대환,32,14,0.000126,0.000067,-8.977302,-9.612992,1.888411


In [ ]:
#
